In [43]:
from pymongo import MongoClient
import pandas as pda
from html import parser
import unicodedata
import re
import requests
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from multiprocessing import Pool
from tqdm import tqdm_notebook as tqdm
from stop_words import get_stop_words

In [54]:
from typing import List

In [18]:
pd.set_option('display.max_columns', None)  
#pd.set_option('display.expand_frame_repr', True)
pd.set_option('max_colwidth', 800)

get data from db hosted locally and make df

In [19]:
client = MongoClient('localhost', 27017)
db = client['kwork']
mycol = db['col2']
all_obj = mycol.find()

In [20]:
df = pd.DataFrame(data=list(all_obj))

In [21]:
df.head()

,_id,name,description,price,max_price,currency,buyer_info,link,time_scraped,time_left
0,1023022,Создание сайта,"Здравствуйте. Требуется создание сайта для компании, оказывающей услуги дизайна, рекламы и продвижения в сети. Желательно на Тильде, т.к. мне будет удобно производить дальнейшее редактирование информации на нём. Сайт необходимо будет наполнить контентом- картинки и текст. Текст нужно будет просто скопировать с другого сайта и внести небольшие поправки с намеком на уникальность. Картинки можно подобрать абстрактные. Оригинальный дизайн не требуется- достаточно будет подобрать типовой шаблонный дизайн. Создание личного кабинета и подключение платежных систем не требуется. Достаточно будет формы &quot; Заказать обратный звонок&quot;.",15000.00,45 000,RUB,"{'buyer_name': 'Ilona_Mois', 'buyer_link': 'https://kwork.ru/user/ilona_mois', 'buyer_status': '', 'projects_offered': '4', 'fulfilled': '0%', 'offers': '0'}",https://kwork.ru/projects/1023022,2020-11-19 19:18:22.807,1 день 23 часа
1,1023019,Съемка познавательных и развивающих игр с детьми,"Здравствуйте!!!Если вы педагог,воспитатель и вы любите заниматься с детишками, то Я ИЩУ именно ВАС!!Что мне нужно?От вас нужно видео занимательных,познавательных и обучающих мини-занятий и игр для детей от 1-5 лет.Видео с монтажом на 5-8 мин.Бюджет-500р/занятие",500.00,,RUB,"{'buyer_name': 'reok1979zx', 'buyer_link': 'https://kwork.ru/user/reok1979zx', 'buyer_status': 'Опытный покупатель', 'projects_offered': '18', 'fulfilled': '22%', 'offers': '0'}",https://kwork.ru/projects/1023019,2020-11-19 19:18:22.831,23 часа 58 минут
2,1023018,Перенести сайт на Yii framework на хостинг IHC.ru,"Задача: \nПеренести сайт kladovaya1.ru (Yii framework) на новый хостинг (IHC.ru), подключить SSL (с помощью бесплатного Let&rsquo;s Encrypt сертификата от IHC), убедиться в полной работоспособности после переноса. \nБудут предоставлены данные для переноса: \nАдмин-панель сайта \nДоступ к FTP текущего сайта \nАрхив с БД текущего сайта \nДоступ к новому хостингу \nНеобходимый результат работы: \nДомен сайта прикреплен к новому хостингу, сайт открывается по https (настроен редирект с http), полностью функционирует, работает доступ в админ-панель. \n(!!!) Ищем специалиста для долгосрочного сотрудничества. После переноса сайта будем планомерно перерабатывать весь сайт в плане юзабилити и SEO",1500.00,4 500,RUB,"{'buyer_name': 'EMarketing', 'buyer_link': 'https://kwork.ru/user/emarketing', 'buyer_status': 'Очень мощный покупатель', 'projects_offered': '90', 'fulfilled': '40%', 'offers': '0'}",https://kwork.ru/projects/1023018,2020-11-19 19:18:22.777,23 часа 57 минут
3,1023015,Эвотор синхронизировать с сайтом,Нужно синхронизировать товарные остатки в программе эвотор с интернет магазином,500.00,1 500,RUB,"{'buyer_name': 'lemon0125', 'buyer_link': 'https://kwork.ru/user/lemon0125', 'buyer_status': '', 'projects_offered': '5', 'fulfilled': '40%', 'offers': '0'}",https://kwork.ru/projects/1023015,2020-11-19 19:18:24.463,23 часа 55 минут
4,1023013,Нужен таргетолог в транспортную компанию,"Добрый день! \nНужен таргетолог по продвижению группы в соц. сети Инстаграм. \nСфера: транспортная компания, грузоперевозки. \nОчень важна эффективная работа и опыт работы! За ценой не постоим. \nПрошу написать ваши предложения по стоимости ваших услуг. \nСпасибо!",12000.00,36 000,RUB,"{'buyer_name': 'Marina_Khromtchenko', 'buyer_link': 'https://kwork.ru/user/marina_khromtchenko', 'buyer_status': '', 'projects_offered': '10', 'fulfilled': '20%', 'offers': '1'}",https://kwork.ru/projects/1023013,2020-11-19 19:18:24.510,1 день 23 часа


### Plan
1. Lower
2. Tokenize
3. Remove stopwords
4. Lemmatize words

In [13]:
m = MorphAnalyzer()

# remove all non-letters from text
regex = re.compile("[А-Яа-яA-z]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

In [ ]:
def lemmatize_word(token: str, pymorphy: MorphAnalyzer = m) -> str:
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text: List[str]) -> List[str]:
    return [lemmatize_word(w) for w in text]

In [12]:
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]

In [53]:
def remove_stopwords(lemmas: List[str], stop_words: List[str] = mystopwords) -> List[str]:
    return [w for w in lemmas if w not in stop_words]

In [29]:
def remove_stopwords(lemmas, stop_words = mystopwords):
    return [w for w in lemmas if w not in stop_words]

For russian language we use two lists of stopwords from different libraries  
(see import ```from nltk.corpus import stopwords``` and ```from stop_words import get_stop_words``` above)

In [44]:
stopwords1 = stopwords.words('russian')
stopwords2 = get_stop_words('ru')
stopwords_all = list(set(stopwords2)|set(stopwords1))

In [48]:
li = ['a', ' bc ', ' cd ', 'fgf']

In [52]:
words_only(df.description[0])

['здравствуйте',
 'требуется',
 'создание',
 'сайта',
 'для',
 'компании',
 'оказывающей',
 'услуги',
 'дизайна',
 'рекламы',
 'и',
 'продвижения',
 'в',
 'сети',
 'желательно',
 'на',
 'тильде',
 'т',
 'к',
 'мне',
 'будет',
 'удобно',
 'производить',
 'дальнейшее',
 'редактирование',
 'информации',
 'на',
 'н',
 'м',
 'сайт',
 'необходимо',
 'будет',
 'наполнить',
 'контентом',
 'картинки',
 'и',
 'текст',
 'текст',
 'нужно',
 'будет',
 'просто',
 'скопировать',
 'с',
 'другого',
 'сайта',
 'и',
 'внести',
 'небольшие',
 'поправки',
 'с',
 'намеком',
 'на',
 'уникальность',
 'картинки',
 'можно',
 'подобрать',
 'абстрактные',
 'оригинальный',
 'дизайн',
 'не',
 'требуется',
 'достаточно',
 'будет',
 'подобрать',
 'типовой',
 'шаблонный',
 'дизайн',
 'создание',
 'личного',
 'кабинета',
 'и',
 'подключение',
 'платежных',
 'систем',
 'не',
 'требуется',
 'достаточно',
 'будет',
 'формы',
 'quot',
 'заказать',
 'обратный',
 'звонок',
 'quot']